In [1]:
import re
import json
import os
import sys

import pandas as pd
print(pd.__version__)

parent_path = os.path.abspath('..')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../../')
sys.path.append(parent_path)

from core.ShoreNet.conf import get_data_path

DATA_PATH = get_data_path()

# load .env file
project_path = os.path.abspath('../../')
dotenv_path = os.path.join(project_path, 'secret', '.env')

from dotenv import load_dotenv
load_dotenv(dotenv_path)

# load project variables
from core.ShoreNet.definitions.variables import VariablesManager

var = VariablesManager()
print(var.data_path)


2.1.4
D:/data/sisi


# 1 Get standard indicator of trust score



# 1.1 choose some docks (top docks for coal events) as standard dock polygon

as the result, choose:

1. coal_dock_id = 30; 舟山港煤炭1号码头 - 北沦港; count = 17029
2. coal_dock_id = 116, 曹妃甸散货码头1号, count = 9094
3. coal_dock_id = 13, 京唐煤炭1号码头, count = 3943
4. coal_dock_id = 137, 防城港码头3（散货）, count = 2277
5. coal_dock_id = 1626, 武汉散货码头2号, count = 2551


In [2]:
from core.ShoreNet.events.generic.tools import load_events_with_dock
from core.ShoreNet.events.generic.tools import load_dock_polygon

# -. load dock polygons
dock_polygon_df = load_dock_polygon(var.engine)
coal_dock_polygon_df = dock_polygon_df[dock_polygon_df['type_id'] == 1]

# -. load events with dock
dock_events_df = load_events_with_dock(year=2023, con=var.engine)
coal_dock_events_df = dock_events_df.loc[dock_events_df['coal_dock_id'].isin(coal_dock_polygon_df['dock_id'])]
# coal_dock_events_df = coal_dock_events_df.apply(lambda row: row if ~row['name'].startswith("dbscan") else None, axis=1)
coal_dock_events_df

,event_id,mmsi,begin_time,end_time,begin_lng,begin_lat,avg_speed,event_categories,coal_dock_id,year,month
9,2023010100002883111541336285010,413362850,1672531213,1672733605,108.357,21.5293,0.000000,low_speed_event,1640,2023,1
13,2023010100002883111641250343010,412503430,1672531228,1673752103,108.329,21.6184,0.000000,low_speed_event,137,2023,1
15,2023010100002883111641350417010,413504170,1672531200,1672538718,108.333,21.6203,0.000000,low_speed_event,137,2023,1
29,2023010100002885111741235584000,412355840,1672531208,1672548769,108.597,21.7073,0.000000,stop_event_poly,173,2023,1
40,2023010100002886109041352038010,413520380,1672531227,1672634081,108.621,19.1011,0.000000,low_speed_event,172,2023,1
...,...,...,...,...,...,...,...,...,...,...,...
2940200,2023123123163003122041384216210,413842162,1704064569,1704066821,120.364,32.0143,0.330970,low_speed_event,198,2023,12
2940203,2023123123172994124741351229000,413512290,1704064678,1704064723,119.419,34.7492,0.348447,stop_event_poly,14,2023,12
2940217,2023123123282994124741330639000,413306390,1704065305,1704065443,119.416,34.7469,0.114233,stop_event_poly,14,2023,12
2940245,2023123123552984128941327223000,413272230,1704066928,1704066933,118.488,38.9152,0.306267,stop_event_poly,158,2023,12


In [3]:
# -. get dock event count, find top 10 docks
top_ten_coal_dock_series = coal_dock_events_df['coal_dock_id'].value_counts().head(50)
top_ten_coal_dock_df = pd.DataFrame(
    {
        'coal_dock_id': top_ten_coal_dock_series.index,
        'count': top_ten_coal_dock_series.values
    }
)
top_ten_coal_dock_df = pd.merge(
    left=top_ten_coal_dock_df,
    right=coal_dock_polygon_df.loc[:, ['dock_id', 'name']],
    left_on='coal_dock_id',
    right_on='dock_id',
    how='left'
)
# top_ten_coal_dock_df.loc[:, ['dock_id', 'name', 'count']]  # choose: [30, 116, 13, 137, 1626]
standard_coal_dock_df = top_ten_coal_dock_df.loc[top_ten_coal_dock_df['dock_id'].isin([30, 116, 13, 137, 1626])]
standard_coal_dock_df

,coal_dock_id,count,dock_id,name
2,30,17029,30,舟山港煤炭1号码头 - 北沦港
5,116,9094,116,曹妃甸散货码头1号
17,13,3943,13,京唐煤炭1号码头
29,1626,2551,1626,武汉散货码头2号
32,137,2277,137,防城港码头3（散货）


## 1.2 calculate some indicator value for each chosen dock

there is no relation between mmsi_nunqie, event_count and avg_duration. each dock has its own distribution of these three values.

In [4]:
# -. calculate some indicator value for each chosen dock
indicator_values = []
for _, row in standard_coal_dock_df.iterrows():
    dock_id = row['dock_id']
    dock_name = row['name']
    for month in range(1, 12+1):
        _month_dock_events = coal_dock_events_df.loc[(coal_dock_events_df['coal_dock_id'] == dock_id) & (coal_dock_events_df['month'] == month)]
        _month_dock_events.loc[:, 'stop_duration'] = _month_dock_events.apply(lambda _row: _row['end_time'] - _row['begin_time'], axis=1)
        _month_dock_events = _month_dock_events.loc[(_month_dock_events['stop_duration'] > 1800) & (_month_dock_events['stop_duration'] < 7 * 24 * 3600)]
        indicator_values.append(
            {
                'dock_id': dock_id,
                'dock_name': dock_name,
                'month': month,
                'mmsi_nunique': _month_dock_events['mmsi'].nunique(),
                'event_count': _month_dock_events.shape[0],
                'avg_duration': _month_dock_events['stop_duration'].mean(),
            }
        )
indicator_values_df = pd.DataFrame(indicator_values)
indicator_values_df

C:\Users\qiu\AppData\Local\Temp\ipykernel_23052\856953730.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _month_dock_events.loc[:, 'stop_duration'] = _month_dock_events.apply(lambda _row: _row['end_time'] - _row['begin_time'], axis=1)
C:\Users\qiu\AppData\Local\Temp\ipykernel_23052\856953730.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _month_dock_events.loc[:, 'stop_duration'] = _month_dock_events.apply(lambda _row: _row['end_time'] - _row['begin_time'], axis=1)
C:\Users\qiu\AppData\Local\Temp\

,dock_id,dock_name,month,mmsi_nunique,event_count,avg_duration
0,30,舟山港煤炭1号码头 - 北沦港,1,214,389,33836.107969
1,30,舟山港煤炭1号码头 - 北沦港,2,338,726,39903.561983
2,30,舟山港煤炭1号码头 - 北沦港,3,421,1046,44211.056405
3,30,舟山港煤炭1号码头 - 北沦港,4,157,317,41479.331230
4,30,舟山港煤炭1号码头 - 北沦港,5,416,891,36728.285073
5,30,舟山港煤炭1号码头 - 北沦港,6,529,1151,35847.305821
6,30,舟山港煤炭1号码头 - 北沦港,7,570,1229,40307.994304
7,30,舟山港煤炭1号码头 - 北沦港,8,313,427,28985.690867
8,30,舟山港煤炭1号码头 - 北沦港,9,426,904,37835.378319
9,30,舟山港煤炭1号码头 - 北沦港,10,422,866,43363.129330


# 1.3 calculate density of effectual area for each dock
1. dbscan events for each dock
2. calculate density of effectual area(cluster_id != -1) for each dock
3. get avg density as trust_score. see how it works.

dbscan parameters:
1. events_count(samples_num) = 80
2. radius = 100m

In [7]:
import numpy as np
from sklearn.cluster import DBSCAN

import plotly.express as px

import plotly.io as pio

indicator_values = []
for _, row in standard_coal_dock_df.iterrows():
    dock_id = row['dock_id']
    dock_name = row['name']
    for month in range(1, 12+1):
        _month_dock_events = coal_dock_events_df.loc[(coal_dock_events_df['coal_dock_id'] == dock_id) & (coal_dock_events_df['month'] == month)]
        _month_dock_events.loc[:, 'stop_duration'] = _month_dock_events.apply(lambda _row: _row['end_time'] - _row['begin_time'], axis=1)
        _month_dock_events = _month_dock_events.loc[(_month_dock_events['stop_duration'] > 1800) & (_month_dock_events['stop_duration'] < 7 * 24 * 3600)]
        print(_month_dock_events.shape)

        coords = _month_dock_events[[var.column_names.lng_column_name, var.column_names.lat_column_name]].values
        db = DBSCAN(
            eps=0.1 / var.geo_param.kms_per_radian,
            min_samples=10,  # convert 0.1m to radian.
            algorithm='ball_tree',
            metric='haversine'
        ).fit(np.radians(coords))
        _month_dock_events.loc[:, 'cluster_id'] = db.labels_
        # print(f"dock name: {dock_name}, before cluster events shape: {_month_dock_events.shape}")
        before_cluster_events_count = _month_dock_events.shape[0]
        _month_dock_events = _month_dock_events.loc[_month_dock_events['cluster_id'] != -1]
        # print(f"dock name: {dock_name}, after cluster events shape: {_month_dock_events.shape}")
        after_cluster_events_count = _month_dock_events.shape[0]
        indicator_values.append(
            {
                'dock_id': dock_id,
                'dock_name': dock_name,
                'month': month,
                'before_cluster_count': before_cluster_events_count,
                'after_cluster_count': after_cluster_events_count
            }
        )

        # Plot the clusters
        fig = px.scatter_mapbox(
            _month_dock_events,
            lat=var.column_names.lat_column_name,
            lon=var.column_names.lng_column_name,
            color='cluster_id',
            title='DBSCAN Clusters',
            color_continuous_scale=px.colors.cyclical.IceFire,
            zoom=5,
            height=600
        )

        fig.update_layout(
            mapbox_style="carto-positron",
            mapbox_center={
                "lat": _month_dock_events[var.column_names.lat_column_name].mean(),
                "lon": _month_dock_events[var.column_names.lat_column_name].mean()
            },
            margin={"r":0,"t":0,"l":0,"b":0},
            height=700, width=1200
        )

        output_file = os.path.join(var.root_path, var.dp_names.output_path, 'html', f'dbscan_clusters_{dock_id}_{month}.html')
        print(output_file)
        pio.write_html(fig, file=output_file, auto_open=False)
        # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        break

indicator_values_df = pd.DataFrame(indicator_values)
indicator_values_df

C:\Users\qiu\AppData\Local\Temp\ipykernel_23052\1758798352.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\qiu\AppData\Local\Temp\ipykernel_23052\1758798352.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\qiu\AppData\Local\Temp\ipykernel_23052\1758798352.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

D:/IdeaProjects/SISI\output\html\dbscan_clusters_30_1.html
D:/IdeaProjects/SISI\output\html\dbscan_clusters_116_1.html
D:/IdeaProjects/SISI\output\html\dbscan_clusters_13_1.html
D:/IdeaProjects/SISI\output\html\dbscan_clusters_1626_1.html
D:/IdeaProjects/SISI\output\html\dbscan_clusters_137_1.html


C:\Users\qiu\AppData\Local\Temp\ipykernel_23052\1758798352.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,dock_id,dock_name,month,before_cluster_count,after_cluster_count
0,30,舟山港煤炭1号码头 - 北沦港,1,389,311
1,116,曹妃甸散货码头1号,1,384,215
2,13,京唐煤炭1号码头,1,112,66
3,1626,武汉散货码头2号,1,264,174
4,137,防城港码头3（散货）,1,80,33
